In [ ]:
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from datetime import datetime
import os.path
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from PIL import Image
import glob
#url = 'http://map1c.vis.earthdata.nasa.gov/wmts-geo/wmts.cgi'
#layer = 'BlueMarble_NextGeneration'
img = plt.imread('/glade/work/jsimkins/bluemarble.png')
img_extent = (-180, 180, -90, 90)

#workdir = ""/Users/james/Documents/Github/esm_lab/MOM6/analysis/output/nwa25/Cheyenne/2021-11-05/"
workdir = "/glade/work/jsimkins/runs-MOM6/old_runs/nwa25_epbl_update_12_2021/"

ds = xr.open_mfdataset(workdir + "20100105.ocean_daily*")
ds_grid = xr.open_dataset("/glade/work/jsimkins/gridInfo/nwa25/land_mask.nc")
var_list = ['sss']

for var in var_list:
    print(var)
    for i in range(0,len(ds.time.values)):
        if os.path.isfile(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png") == False:
            figure = plt.figure(figsize=(12,12))
            left = 0.01
            bottom = 0.01
            width = 0.99
            height = 0.99
            ax = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
            if var=='sst':
                im = ax.pcolormesh(ds.xh.values, ds.yh.values, ds.sst[i].values, cmap='turbo', vmin=0, vmax=30)
                cb = plt.colorbar(im, ticks=[5,10,15,20,25], shrink=0.6)
                cb.set_label('Sea Surface Temperature (deg C))', color='w', fontsize=12)
                plt.title(str("MOM6 NWA25 Sea Surface Temperature - " + str(ds.time[i].values)), fontsize=16, color='w')
            if var=='ssh050':
                im = ax.pcolormesh(ds.x.values, ds.yh.values, ds.ssh[i].values, cmap='turbo', vmin=-0.50, vmax=0.50)
                cb = plt.colorbar(im, ticks=[-0.25,0,0.25], shrink=0.6)
                cb.set_label('Sea Surface Height (m)', color='w', fontsize=12)
                plt.title(str("MOM6 NWA25 Sea Surface Height - " + str(ds.time[i].values)), fontsize=16, color='w')
            if var=='sss':
                im = ax.pcolormesh(ds_grid.x.values, ds_grid.y.values, ds.sss[i].values, cmap='turbo', vmin=30, vmax=40)
                cb = plt.colorbar(im, ticks=[32,34,36,38], shrink=0.6)
                cb.set_label('Sea Surface Salinity (psu)', color='w', fontsize=12)
                plt.title(str("MOM6 NWA25 Sea Surface Salinity - " + str(ds.time[i].values)), fontsize=16, color='w')
            ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
            gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
            gl.top_labels = False
            gl.left_labels = False
            gl.right_labels=True
            gl.bottom_labels=True
            gl.xlines = True
            gl.ylines = True
            gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
            gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
            gl.xformatter = LONGITUDE_FORMATTER
            gl.yformatter = LATITUDE_FORMATTER
            gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
            gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
            cb.ax.yaxis.set_tick_params(color='w')
            plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
            outfile = str(workdir + "/images/" + var + datetime.strptime(str(ds.time[i].values), "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M") + ".png")
            plt.savefig(outfile, facecolor='black', bbox_inches='tight', format='png', dpi=300)
            plt.close()
            
    img_names = sorted(glob.glob(workdir + "/images/" + var + "*.png"))
    dur_vals = []
    for i in range(0,len(img_names)):
        if i != len(img_names)-1:
            dur_vals.append(200)
    dur_vals.append(3000)

    img, *imgs = [Image.open(f) for f in img_names]
    img.save(fp=workdir + var + '.gif', format='GIF', append_images=imgs,
         save_all=True, duration=dur_vals, loop=0)
